In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random

In [ ]:
#Helper functions
def gradient_penalty(X,X_fake,epsilon):
    inputs_gp = epsilon*X + ((1-epsilon)*X_fake)
    disc_inputs_gp = discriminator(inputs_gp,reuse=True)
    grad_disc_inputs = tf.gradients(disc_inputs_gp,inputs_gp)
    penalty = tf.square(tf.norm(grad_disc_inputs)-1)
    return penalty
    
def data():   
    scale = 2.
    centers = [
        (1,0),
        (-1,0),
        (0,1),
        (0,-1),
        (1./np.sqrt(2), 1./np.sqrt(2)),
        (1./np.sqrt(2), -1./np.sqrt(2)),
        (-1./np.sqrt(2), 1./np.sqrt(2)),
        (-1./np.sqrt(2), -1./np.sqrt(2))
    ]
    centers = [(scale*x,scale*y) for x,y in centers]
    while True:
        dataset = []
        for i in range(batch_size):
            point = np.random.randn(2)*.02
            center = random.choice(centers)
            point[0] += center[0]
            point[1] += center[1]
            dataset.append(point)
        dataset = np.array(dataset, dtype='float32')
        dataset /= 1.414 # stdev
        yield dataset



def generate_image(true_dist, samples, disc_map, Range = 3, N = 128):
            plt.clf()
            x = y = np.linspace(-Range, Range, N)
            plt.contour(x,y,disc_map.reshape((len(x), len(y))).transpose())
            plt.scatter(true_dist[:, 0], true_dist[:, 1], c='orange',  marker='+')
            plt.scatter(samples[:, 0],    samples[:, 1],    c='green', marker='+')
            plt.show()

In [ ]:
def generator(Z, output_dim = 1, h_dim = 32, reuse = False):
    with tf.variable_scope('generator',reuse=reuse):
        hidden1 = tf.layers.dense(Z,h_dim)
        hidden1 = tf.nn.relu(hidden1)
        
        hidden2 = tf.layers.dense(hidden1,h_dim)
        hidden2 = tf.nn.relu(hidden2)
        
        hidden3 = tf.layers.dense(hidden2,h_dim)
        hidden3 = tf.nn.relu(hidden3)
        
        output = tf.layers.dense(hidden3,output_dim)
        return output        

In [ ]:
def discriminator(X, h_dim = 512, reuse = False):
    with tf.variable_scope('discriminator',reuse=reuse):
        hidden1 = tf.layers.dense(X,h_dim)
        hidden1 = tf.nn.relu(hidden1)
        
        hidden2 = tf.layers.dense(hidden1,h_dim)
        hidden2 = tf.nn.relu(hidden2)
        
        hidden3 = tf.layers.dense(hidden2,h_dim)
        hidden3 = tf.nn.relu(hidden3)
        
        logits = tf.layers.dense(hidden3,1)
        return logits 

In [ ]:
def generator_loss(logits_fake):
    return -tf.reduce_mean(logits_fake)

def discriminator_loss(logits_real, logits_fake, penalty, lambd = 10):
    return tf.reduce_mean(logits_fake) - tf.reduce_mean(logits_real) + lambd*tf.reduce_mean(penalty) 

In [ ]:
def training_op(disc_loss, gen_loss, learning_rate = 1e-4, beta1 = 0.0, beta2 = 0.9):
    disc_var_list=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='discriminator')
    gen_var_list=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='generator')
    
    disc_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=beta1,beta2=beta2)
    gen_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=beta1,beta2=beta2)
    
    disc_train_op = disc_optimizer.minimize(disc_loss,var_list=disc_var_list)
    gen_train_op = disc_optimizer.minimize(gen_loss,var_list=gen_var_list)
    return disc_train_op,gen_train_op

In [ ]:
def input_data(z_dim = 2, output_dim = 1, batch_size = 1024):
    X = tf.placeholder(dtype=tf.float32, shape = [None, output_dim])
    Z = tf.random_normal([2, z_dim])
    Z_ = tf.random_uniform([batch_size,z_dim])
    return X,Z,Z_

In [ ]:
#Hyperparmetrs
h_dim = 512
z_dim = 2
output_dim = 2
batch_size = 256
learning_rate = 1e-3
beta1 = 0.0
beta2 = 0.9
lambd = 10
epochs = 5001
batch_it = 5

In [ ]:
g1 = tf.Graph()
with g1.as_default() as graph:
    X,Z,Z_ = input_data(z_dim, output_dim, batch_size)
    X_fake = generator(Z, output_dim=output_dim, h_dim=h_dim)
    X_fake_ = generator(Z_, output_dim=output_dim, h_dim=h_dim, reuse=True)
    epsilon = tf.random_uniform([batch_size,1],minval=0,maxval=1)

    logits_real = discriminator(X,h_dim=h_dim)
    logits_fake = discriminator(X_fake,h_dim=h_dim,reuse=True)
    penalty = gradient_penalty(X,X_fake_,epsilon)

    disc_loss = discriminator_loss(logits_real,logits_fake,penalty,lambd)
    gen_loss = generator_loss(logits_fake)

    disc_train_op,gen_train_op = training_op(disc_loss, gen_loss, learning_rate=learning_rate, beta1=beta1, beta2=beta2)

In [ ]:
N = 128
Range = 3

points = np.zeros((N, N, 2), dtype='float32')
points[:,:,0] = np.linspace(-Range, Range, N)[:,None]
points[:,:,1] = np.linspace(-Range, Range, N)[None,:]
points = points.reshape((-1,2))
print(points.shape)

In [ ]:
with tf.Session(graph=graph) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    data_gen = data()
    gen_cost = 0.0
    for epoch in range(epochs):
        if epoch > 0:
            _ = sess.run(gen_train_op)         
        for i in range(batch_it):
            d = next(data_gen)
            feed_dict = {X:d}
            disc_cost,gen_cost,_ = sess.run([disc_loss,gen_loss,disc_train_op],feed_dict={X:d})
        if epoch %50 == 0:
            x_fake = sess.run(X_fake)
            print("====")
            print("Iteration: {}/{}".format(epoch,epochs-1))
            print("Discriminator loss: {}".format(disc_cost))
            print("Generator loss: {}".format(gen_cost))
        if epoch %100 ==0:
            samples, disc_map = sess.run([X_fake, logits_real], feed_dict={X:points})
            generate_image(d, samples, disc_map, Range=Range, N=N)